## Temporal Filtering of UNOSAT Damage Assessment Data

This notebook automates the extraction of the temporally closest damage observation per site from a multi-date UNOSAT damage assessment dataset. The source data, structured as a GeoJSON file, includes point-based damage features with repeated temporal observations across multiple fields (`SensorDate`, `SensorDate_2`, ..., `SensorDate_9`). The objective is to isolate the date of observation that is closest to a given reference date — in this case, **February 18, 2025** — for each individual damage point.

---

### Methodology

1. **Data Loading and Inspection**
   - The notebook loads the GeoJSON file using `geopandas`, identifies all date-related columns, and ensures they are parsed as datetime objects.

2. **Closest Date Identification**
   - For each feature, the script:
     - Iterates over all valid observation dates
     - Computes the absolute time difference from the target date
     - Selects the date with the smallest difference
   - The result is stored in a new column, `closest_date`.

3. **Temporal Filtering (Optional)**
   - A subset of features can be extracted that fall within a specified time window around the target date (e.g., ±1 day).

4. **Data Export**
   - The filtered dataset is exported as a new GeoJSON file for visualization or further analysis in GIS environments.

---

### Applications

This workflow is useful for:
- **Conflict damage monitoring**
- **Time-sensitive impact analysis**
- Selecting representative damage records for a known **reference event or strike date**
- Reducing multi-temporal complexity into a single-date representation for mapping and statistics

---

### Technologies Used

- Python 3
- `geopandas`, `pandas`, `datetime`
- Jupyter Notebook or Google Colab

---

The notebook ensures reproducible and scalable preprocessing of UNOSAT damage data for spatiotemporal filtering, enabling targeted analysis and integration with field observations or remote sensing datasets.


In [1]:
!pip install geopandas pandas

import geopandas as gpd
import pandas as pd
from datetime import datetime

In [3]:
from google.colab import drive
drive.mount('/content/drive')

gdf = gpd.read_file("/content/drive/MyDrive/UNOSAT_damage_pts.geojson")

Mounted at /content/drive


In [4]:
date_columns = [col for col in gdf.columns if col.startswith("SensorDate")]
gdf[date_columns].head()

,SensorDate,SensorDate_2,SensorDate_3,SensorDate_4,SensorDate_5,SensorDate_6,SensorDate_7,SensorDate_8,SensorDate_9
0,NaT,2023-11-07 00:00:00+00:00,2023-11-26 00:00:00+00:00,2024-01-07 00:00:00+00:00,2024-02-29 00:00:00+00:00,2024-03-31 00:00:00+00:00,2024-05-03 00:00:00+00:00,2024-07-06 00:00:00+00:00,2024-09-03 00:00:00+00:00
1,NaT,NaT,NaT,2024-01-07 00:00:00+00:00,2024-02-29 00:00:00+00:00,2024-03-31 00:00:00+00:00,2024-05-03 00:00:00+00:00,2024-07-06 00:00:00+00:00,2024-09-03 00:00:00+00:00
2,NaT,NaT,NaT,2024-01-07 00:00:00+00:00,2024-02-29 00:00:00+00:00,2024-03-31 00:00:00+00:00,2024-05-03 00:00:00+00:00,2024-07-06 00:00:00+00:00,2024-09-03 00:00:00+00:00
3,NaT,2023-11-07 00:00:00+00:00,2023-11-26 00:00:00+00:00,2024-01-07 00:00:00+00:00,2024-02-29 00:00:00+00:00,2024-03-31 00:00:00+00:00,2024-05-03 00:00:00+00:00,2024-07-06 00:00:00+00:00,2024-09-03 00:00:00+00:00
4,NaT,NaT,NaT,2024-01-07 00:00:00+00:00,2024-02-29 00:00:00+00:00,2024-03-31 00:00:00+00:00,2024-05-03 00:00:00+00:00,2024-07-06 00:00:00+00:00,2024-09-03 00:00:00+00:00


In [5]:
for col in date_columns:
    gdf[col] = pd.to_datetime(gdf[col], errors='coerce')

In [7]:
target_date = pd.to_datetime("2025-02-18").tz_localize('UTC')

def closest_date(row):
    valid_dates = [row[col] for col in date_columns if pd.notnull(row[col])]
    if not valid_dates:
        return pd.NaT
    return min(valid_dates, key=lambda d: abs(d - target_date))

gdf["closest_date"] = gdf.apply(closest_date, axis=1)

In [8]:
filtered = gdf[abs(gdf["closest_date"] - target_date) <= pd.Timedelta(days=1)]

In [9]:
filtered.to_file("UNOSAT_closest_to_feb18.geojson", driver="GeoJSON")